In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from gzip import open

from numpy import nan
from pandas import DataFrame

In [ ]:
file_path = "/home/kwat/CarrotCake/garden/data/medulloblastoma/northcott_et_al/GSE37382_family.soft.gz"

In [ ]:
def next_line(io):

    return io.readline().decode()


with open(file_path) as io:

    platform_table = []

    samples = {}

    for line in io:

        line = line.decode()

        if line == "!platform_table_begin\n":

            line = next_line(io)

            while line != "!platform_table_end\n":

                platform_table.append(line.rstrip().split(sep="\t"))

                line = next_line(io)

        if line.startswith("^SAMPLE"):

            sample = line.rstrip().split(sep=" = ")[1]

            samples[sample] = {"table": []}

            line = next_line(io)

            while line and not line.startswith("^SAMPLE"):

                if line == "!sample_table_begin\n":

                    line = next_line(io)

                    while line != "!sample_table_end\n":

                        samples[sample]["table"].append(line.rstrip().split(sep="\t"))

                        line = next_line(io)

                else:

                    try:

                        key, value = line.rstrip().split(sep=" = ")

                    except ValueError:

                        line = next_line(io)

                        continue

                    samples[sample][key] = value

                line = next_line(io)

platform_table = DataFrame(data=platform_table)

platform_table.columns = platform_table.iloc[0]

assert not platform_table.columns.has_duplicates

platform_table.drop(0, inplace=True)

platform_table.set_index("ID", inplace=True)

assert not platform_table.index.has_duplicates

for dict_ in samples.values():

    table = DataFrame(dict_["table"])

    table.columns = table.iloc[0]

    assert not table.columns.has_duplicates

    table.drop(0, inplace=True)

    table.set_index("ID_REF", inplace=True)

    assert not table.index.has_duplicates

    dict_["table"] = table

In [ ]:
def gene_assignment(str_):

    if "//" in str_:

        return str_.split("//")[1].strip()


for column, function in (
    ("gene_symbol", None),
    ("oligoset_genesymbol", None),
    ("ilmn_gene", None),
    ("gene", None),
    ("gene_assignment", gene_assignment),
):

    if column in platform_table:

        series = platform_table[column]

        if function is not None:

            series = series.apply(function)

        id_gene = series.fillna(nan).to_dict()